In [1]:
import os, sys, logging, importlib, math, time, subprocess
import rasterio, affine

import geopandas as gpd
import pandas as pd

from zipfile import ZipFile
from affine import Affine
from rasterio import features
from rasterio.mask import mask
from rasterio.features import rasterize

import GEP

'''prints the time along with the message'''
def tPrint(s):
    print("%s\t%s" % (time.strftime("%H:%M:%S"), s))

# Mapping electrification
Code to download results from S3

We need to convert certain fields in the results to a raster; these fields need to be calculated 

1. Population already electrified 2030  
    a. Rasterize field **ElecPop**
2. New population electrified by grid 2030  
    a. Filter data where **FinalElecCode2030** == 1.0  
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** == 99.0  
    c. Combine **NewConnections2030** with **NewConnections2025**  
3. New population electrified by off grid 2030  
    a. Filter data where **FinalElecCode2030** != 1.0  
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** != **FinalElecCode2030**  
    c. Combine **NewConnections2030** with **NewConnections2025**  

In [2]:
# delete all existing rasterized results
'''all_rasters = []
for root, dirs, files in os.walk('/media/gost/DATA1/GEP/Clusters'):
    for f in files:
        if "rasterized_gep" in f:
            all_rasters.append(os.path.join(root, f))
for f in all_rasters:
    os.remove(f)'''

'all_rasters = []\nfor root, dirs, files in os.walk(\'/media/gost/DATA1/GEP/Clusters\'):\n    for f in files:\n        if "rasterized_gep" in f:\n            all_rasters.append(os.path.join(root, f))\nfor f in all_rasters:\n    os.remove(f)'

In [3]:
resolution = 0.01 #Resolution of output raster in degrees
elecRate = 0.90   #settlement electrification rate to be considered electrified
year = 2030       #year of analysis 
cluster_folder = '/media/gost/DATA1/GEP/Clusters'
scenario_base = '/media/gost/DATA1/GEP/Scenarios'
raster_folder = '/media/gost/DATA1/GEP/rasterized_countries'
vrt_folder = '/media/gost/DATA1/GEP/GEP_VRTs'


In [4]:
importlib.reload(GEP)
scenario = "0_0_0_0_1_0"
scenario_folder = os.path.join(scenario_base, f'{scenario}')
# Identify countries to process
zipFiles = []
processedCountries = []
for root, folders, files in os.walk(scenario_folder):    
    processedCountries.append([f for f in folders])
    for f in files:
        if f[-4:] == ".csv":
            zipFiles.append(f[:-4])
processedCountries = [item[:4] for item in zipFiles]
zipFiles #list of country/scenarios to process

['gh-2-0_0_0_0_1_0',
 'mw-2-0_0_0_0_1_0',
 'ao-2-0_0_0_0_1_0',
 'bd-2-0_0_0_0_1_0',
 'bf-2-0_0_0_0_1_0',
 'bi-2-0_0_0_0_1_0',
 'bj-2-0_0_0_0_1_0',
 'bw-2-0_0_0_0_1_0',
 'cd-2-0_0_0_0_1_0',
 'cf-2-0_0_0_0_1_0',
 'cg-2-0_0_0_0_1_0',
 'ci-2-0_0_0_0_1_0',
 'cm-2-0_0_0_0_1_0',
 'dj-2-0_0_0_0_1_0',
 'er-2-0_0_0_0_1_0',
 'et-2-0_0_0_0_1_0',
 'fm-2-0_0_0_0_1_0',
 'ga-2-0_0_0_0_1_0',
 'gm-2-0_0_0_0_1_0',
 'gn-2-0_0_0_0_1_0',
 'gq-2-0_0_0_0_1_0',
 'gw-2-0_0_0_0_1_0',
 'hn-2-0_0_0_0_1_0',
 'ht-2-0_0_0_0_1_0',
 'ke-2-0_0_0_0_1_0',
 'kh-2-0_0_0_0_1_0',
 'km-2-0_0_0_0_1_0',
 'lr-2-0_0_0_0_1_0',
 'ls-2-0_0_0_0_1_0',
 'mg-2-0_0_0_0_1_0',
 'ml-2-0_0_0_0_1_0',
 'mm-2-0_0_0_0_1_0',
 'mr-2-0_0_0_0_1_0',
 'mz-2-0_0_0_0_1_0',
 'na-2-0_0_0_0_1_0',
 'ne-2-0_0_0_0_1_0',
 'ng-2-0_0_0_0_1_0',
 'ni-2-0_0_0_0_1_0',
 'pg-2-0_0_0_0_1_0',
 'pk-2-0_0_0_0_1_0',
 'rw-2-0_0_0_0_1_0',
 'sb-2-0_0_0_0_1_0',
 'sd-2-0_0_0_0_1_0',
 'sl-2-0_0_0_0_1_0',
 'sn-2-0_0_0_0_1_0',
 'so-2-0_0_0_0_1_0',
 'ss-2-0_0_0_0_1_0',
 'st-2-0_0_0_

In [5]:
importlib.reload(GEP)
# Loop through all countries and rasterize
minPop = 100
for country in processedCountries:
    xx_so = GEP.gep_results(country[:4], 
                scenariosFolder=scenario_base,
                clustersFolder = cluster_folder,
                scenario=scenario)
    outFile = os.path.join(raster_folder, f"{country[:2]}_rasterized_gep_{resolution}_{scenario}_{year}_gt{minPop}.tif")    
    if not os.path.exists(outFile):
        tPrint(f"Processing {country}")    
        res = xx_so.join_results()
        res = res.loc[res['Pop2030'] > minPop,]
        # Identify already electrified settlements and set value to 98
        res['already_electrified'] = (res['ElecPopCalib'] / res['Pop2020']) > elecRate
        res.loc[res['already_electrified'] == 1, 'FinalElecCode%s' % year] = 98
        xx_so.rasterize_results(res, outFile, res=resolution, field='FinalElecCode%s' % year)
    else:
        tPrint(f"{country} already rasterized")    

10:11:36	gh-2 already rasterized
10:11:36	mw-2 already rasterized
10:11:36	ao-2 already rasterized
10:11:36	bd-2 already rasterized
10:11:36	bf-2 already rasterized
10:11:36	bi-2 already rasterized
10:11:36	bj-2 already rasterized
10:11:36	bw-2 already rasterized
10:11:36	cd-2 already rasterized
10:11:36	cf-2 already rasterized
10:11:36	cg-2 already rasterized
10:11:36	ci-2 already rasterized
10:11:36	cm-2 already rasterized
10:11:36	dj-2 already rasterized
10:11:36	er-2 already rasterized
10:11:36	et-2 already rasterized
10:11:36	fm-2 already rasterized
10:11:36	ga-2 already rasterized
10:11:36	gm-2 already rasterized
10:11:36	gn-2 already rasterized
10:11:36	gq-2 already rasterized
10:11:36	gw-2 already rasterized
10:11:36	hn-2 already rasterized
10:11:36	ht-2 already rasterized
10:11:36	ke-2 already rasterized
10:11:36	kh-2 already rasterized
10:11:36	km-2 already rasterized
10:11:36	lr-2 already rasterized
10:11:36	ls-2 already rasterized
10:11:36	mg-2 already rasterized
10:11:36	m

In [18]:
### Run a series of GDAL commands to create No Data rasters and stack them
command_file = os.path.join(vrt_folder, f'create_vrt_{resolution}_{scenario}_{year}.sh')
with open(command_file, 'w') as vrt_file:
    vrt_file.write('#!/bin/sh \n')
    # Convert 0 in data to no data
    outFolder = raster_folder
    allImages = []
    for root, folders, files in os.walk(outFolder):
        for f in files:
            if "%s_%s_%s_gt%s.tif" % (resolution, scenario, year, minPop) in f and f.endswith(".tif"):
                allImages.append(os.path.join(root, f))

    for inFile in allImages:
        outFile = inFile.replace(".tif", ".tif")
        vrt_file.write(f'gdal_translate -of GTiff -a_nodata 0 {inFile} {outFile}')
        vrt_file.write("\n")
    vrt_file.write(f'find {raster_folder} -name "*{resolution}_{scenario}_{year}_gt{minPop}_noData.tif" > tifFiles.txt')
    vrt_file.write("\n")
    vrt_file.write(f"gdalbuildvrt -input_file_list tifFiles.txt {vrt_folder}/combined_gep_rasterized_{resolution}_{scenario}_{year}.vrt")
    vrt_file.write("\n")
subprocess.call(command_file)

# DEbugging